Sentimental analysis start from here

In [ ]:
%tensorflow_version 2.x
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
#look at one review
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

In [ ]:
#pre processing coz our data have not same size
#if the review is greater than 250 words then trim off the extra words
#if the review is less than 250 words add the necessary amount of's to make it equal to 250.
#but do not worry keras has a function that can do this for us
train_data = sequence.pad_sequences(train_data,MAXLEN)
test_data = sequence.pad_sequences(test_data,MAXLEN)

In [ ]:
len(train_data[1])

250

Now its time to create the model. We will use a word embedding layer as the first layer in our model and add a
LSTM layer afterwords that feeds into a dense node to get our prediction sentiment
32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like

In [ ]:
#creating a model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE,32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1,activation = "sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Training

In [ ]:
model.compile(loss ="binary_crossentropy",optimizer ="rmsprop",metrics = ['acc'])
history = model.fit(train_data,train_labels,epochs =10, validation_split = 0.2)

Epoch 1/10
625/625 [==============================] - 40s 61ms/step - loss: 0.4583 - acc: 0.7724 - val_loss: 0.3095 - val_acc: 0.8718
Epoch 2/10
625/625 [==============================] - 36s 58ms/step - loss: 0.2625 - acc: 0.8992 - val_loss: 0.3472 - val_acc: 0.8698
Epoch 3/10
625/625 [==============================] - 36s 58ms/step - loss: 0.2048 - acc: 0.9230 - val_loss: 0.2815 - val_acc: 0.8880
Epoch 4/10
625/625 [==============================] - 36s 58ms/step - loss: 0.1638 - acc: 0.9398 - val_loss: 0.3019 - val_acc: 0.8834
Epoch 5/10
625/625 [==============================] - 36s 58ms/step - loss: 0.1364 - acc: 0.9527 - val_loss: 0.3247 - val_acc: 0.8882
Epoch 6/10
625/625 [==============================] - 37s 59ms/step - loss: 0.1122 - acc: 0.9629 - val_loss: 0.3458 - val_acc: 0.8838
Epoch 7/10
625/625 [==============================] - 36s 58ms/step - loss: 0.0986 - acc: 0.9690 - val_loss: 0.3567 - val_acc: 0.8848
Epoch 8/10
625/625 [==============================] - 36s 58ms

In [ ]:
#well evaluate the model on our training data to see how well it performs
results = model.evaluate(test_data,test_labels)
print(results)

782/782 [==============================] - 11s 14ms/step - loss: 0.4200 - acc: 0.8633
[0.420030415058136, 0.863319993019104]


#Making prediction

Now lets use our network to make predictions on our own reviews.
Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it.
To do that well load the encodings from the dataset and use them to encode our own data.

In [ ]:
from tensorflow import keras
word_index = imdb.get_word_index()
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens],MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [ ]:
#while were at it lets make a decode function
reverse_word_index = {value:key for (key,value) in word_index.items()}
def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num]+ " "
  return text[:-1]


In [ ]:
print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
#its time to make a prediction
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])
positive_review = "That movie was so awesome! I really loved it and would watch it again because it was amazingly great"
predict(positive_review)
negative_review = "that movie sucked. I hated it and wouldn't watch it again. was one of the worst things I've ever watched"
predict(negative_review)


1/1 [==============================] - 0s 20ms/step
[0.9003098]
1/1 [==============================] - 0s 20ms/step
[0.24089403]


In [ ]:
my_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
for i in my_list[0:5]:
  print(i)


0
1
2
3
4


In [ ]:
class Node:
  def __init__(self,data):
    self.data = data
    self.next = None
class LinkedList:
  def __init__(self):
    self.head = None
  def delt(self,data):
    c = self.head
    if c and c.data == data:
      self.head = c.next
      c= None
      return
    p = None
    while c and c.data!= data:
      p = c
      c = c.next
    if c is None:
      print("Linked list is empty")
      return
    p.next = c.next
    c= None


  def append(self,data):
    new_node = Node(data)
    if self.head is None:
      self.head = new_node
      return
    last_node = self.head
    while last_node.next:
      last_node = last_node.next
    last_node.next = new_node
  def print(self):
    current = self.head
    while current:
      print("->",current.data)
      current = current.next

obj = LinkedList()
obj.append(2)
obj.append(3)
obj.append(5)
obj.print()
print("after deletion")
obj.delt(5)
obj.print()

after deletion
Linked list is empty


In [ ]:
k=[1,2,3,3,2,2,4]
l=[]
for i in range(len(k)):
  if k[i] in l:
    continue
  else:
    l.append(k[i])

In [ ]:
print(l)

[1, 2, 3, 4]
